In [5]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras import layers
import datag
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [2]:
# from tensorflow.keras.applications import VGG16
# from tensorflow.keras.models import Model

In [6]:
path_dataset = 'D:/GitHub/reconocieminto-facial-con-python/dataset/'
nombres_directorios = datag.extraer_nombres(path_dataset)
print(nombres_directorios)

etiquetas = []
for index, directorio in enumerate(nombres_directorios):
    nombres_archivos = datag.extraer_nombres(path_dataset + directorio)
    total_archivos = len(nombres_archivos)
    for _ in range(total_archivos):
        etiquetas.append(index)
print(f'total etiquetas: {len(etiquetas)}')

imagenes = [] 
for directorio in nombres_directorios:
    nombres_archivos = datag.extraer_nombres(path_dataset + directorio)
    for nombre_archivo in nombres_archivos:
        img_array = cv2.imread(path_dataset + directorio + '/' + nombre_archivo, cv2.IMREAD_GRAYSCALE)
        img_array = img_array / 255 # normalizo
        imagenes.append(img_array)
print(f'total datos: {len(imagenes)}')

['Abraham_Danilo_Miranda_Lopez', 'Alejandro_Benjamin_Rocano_Lopez', 'Alex_Ignacio_Tigselema_Pacheco', 'andres', 'Andres_Patricio_Tapia_Gonzalez', 'Bryan_Eduardo_Martinez_Nunez', 'Christian_Marcelo_Jaramillo_Collazo', 'Diana_Jazmin_Pinchao_Fante', 'Diego_Fernando_Cata_Saltos', 'Edison_Orlando_Lopez_Galarza', 'Jeniffer_Paulina_Yaguana_Caraguay', 'Jonathan_Jose_Pazmino_Fiallos', 'Luis_Fernando_Zerna_Ramos', 'Marco_Farid_Ruano_Caicedo', 'mateo', 'Pablo_Jhoel_Puetate_Mier', 'Silvia_Esthefania_Villacres_Chico', 'Thais_Eliana_Armijos_Troya']
total etiquetas: 18000


total datos: 18000


In [7]:
xtrain, xtest, ytrain, ytest = train_test_split(imagenes, etiquetas, random_state=10)

In [8]:
xtrain = np.array(xtrain)
ytrain = np.array(ytrain)

xtest = np.array(xtest)
ytest = np.array(ytest)

print(xtrain.shape)

(13500, 100, 100)


In [ ]:
# base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# # Construir el modelo VGG16 completo agregando capas adicionales
# x = base_model.output
# x = layers.GlobalAveragePooling2D()(x)
# x = layers.Dense(4096, activation='relu')(x)
# x = layers.Dense(4096, activation='relu')(x)
# output = layers.Dense(len(nombres_directorios), activation='softmax')(x)  # Reemplaza 'num_classes' con el número de clases de tu problema

# # Crear el modelo final
# modelo = Model(inputs=base_model.input, outputs=output)


In [10]:
modelo = tf.keras.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(xtrain[0].shape[0],xtrain[0].shape[0],1)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    layers.Dropout(0.5),
    layers.Flatten(),
    
    layers.Dense(256, activation=tf.nn.relu),
    layers.Dense(512, activation=tf.nn.relu),
    layers.Dense(len(nombres_directorios), activation=tf.nn.softmax)
])
modelo.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])
modelo.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 98, 98, 32)        320       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 49, 49, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 23, 23, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 21, 21, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 10, 10, 128)      

In [11]:
modelo.fit(xtrain, ytrain, epochs=12, batch_size=32, shuffle=True)

Epoch 1/12
422/422 [==============================] - 176s 408ms/step - loss: 1.0770 - accuracy: 0.6389
Epoch 2/12
422/422 [==============================] - 170s 403ms/step - loss: 0.2685 - accuracy: 0.9144
Epoch 3/12
422/422 [==============================] - 166s 392ms/step - loss: 0.1550 - accuracy: 0.9493
Epoch 4/12
422/422 [==============================] - 164s 389ms/step - loss: 0.0944 - accuracy: 0.9691
Epoch 5/12
422/422 [==============================] - 161s 382ms/step - loss: 0.0805 - accuracy: 0.9739
Epoch 6/12
422/422 [==============================] - 167s 397ms/step - loss: 0.0640 - accuracy: 0.9790
Epoch 7/12
422/422 [==============================] - 161s 382ms/step - loss: 0.0542 - accuracy: 0.9816
Epoch 8/12
422/422 [==============================] - 116s 275ms/step - loss: 0.0459 - accuracy: 0.9845
Epoch 9/12
422/422 [==============================] - 117s 278ms/step - loss: 0.0537 - accuracy: 0.9839
Epoch 10/12
422/422 [==============================] - 120s 285m

In [13]:
predicciones = modelo.predict(xtest)
y_pred = np.argmax(predicciones, axis=1)

exactitud = accuracy_score(ytest, y_pred)
precision = precision_score(ytest, y_pred, average='macro')
sensibilidad = recall_score(ytest, y_pred, average='macro')
puntuacion_f1 = f1_score(ytest, y_pred, average='macro')

print("Exactitud:", exactitud)
print("Precisión:", precision)
print("Sensibilidad:", sensibilidad)
print("Puntuación F1:", puntuacion_f1)

141/141 [==============================] - 9s 63ms/step
Exactitud: 0.9777777777777777
Precisión: 0.9777285075888541
Sensibilidad: 0.9775741881311958
Puntuación F1: 0.9774287186505615


descgargar el modelo

In [14]:
modelo.save('modelo_cnn')
modelo.save_weights('pesos_cnn.h5')

INFO:tensorflow:Assets written to: modelo_cnn\assets


INFO:tensorflow:Assets written to: modelo_cnn\assets
